In [1]:
# Dependencies and Setup
import requests
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import os
import csv

# Import the API key
from api_key import bls_api_key

# The below code is from the BLS API Guide I found online from BD Econ

It's the website I sent on Slack

The only thing the needs changing is the date, it looks like the INDEX, but DATE should be a column and datetime object type so we can plot and merge it with the other data points

In [2]:
# DO NOT USE SERIES ID THAT ENDS WITH Q - it stands for QUARTER and messes up the data

url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
# API key in config.py which contains: bls_key = 'key'
key = '?registrationkey={}'.format(bls_api_key)
# Series stored as a dictionary
series_dict = {
    'LNU04032231': '(Unadj) Unemployment Rate - Construction Industry, Private Wage and Salary Workers)',
    'LNU04034236': '(Unadj) Unemployment Rate - Nonagricultural Private Wage and Salary Workers, Educational Services',
    'LNU04034239': '(Unadj) Unemployment Rate - Nonagricultural Private Wage and Salary Workers, Health Care and Social Assistance',
    'LNU04032230': '(Unadj) Unemployment Rate - Mining, quarrying, and oil and gas extraction, Nonagricultural Private Wage and Salary Workers',
    'LNU04034202': '(Unadj) Unemployment Rate - Nonagricultural Private Wage and Salary Workers, Finance',
    'LNU04034211': '(Unadj) Unemployment Rate - Nonagricultural Private Wage and Salary Workers, Real Estate',
    'LNU04034205': '(Unadj) Unemployment Rate - Nonagricultural Private Wage and Salary Workers, Insurance',
    'LNU04034199': '(Unadj) Unemployment Rate - Nonagricultural Private Wage and Salary Workers, Finance and Insurance'}
# Start year and end year
dates = ('2018', '2019')
#dates = ('2018', '2019', '2020', '2021', '2022')

In [3]:
headers = {'Content-type': 'application/json'}
# Submit the list of series as data
data = json.dumps({
    "seriesid": list(series_dict.keys()),
    "startyear": dates[0],
    "endyear": dates[1]})
# Post request for the data
p = requests.post(
    '{}{}'.format(url, key),
    headers=headers,
    data=data).json()['Results']['series']

print(p)

[{'seriesID': 'LNU04032231', 'data': [{'year': '2019', 'period': 'M12', 'periodName': 'December', 'value': '5.0', 'footnotes': [{}]}, {'year': '2019', 'period': 'M11', 'periodName': 'November', 'value': '4.4', 'footnotes': [{}]}, {'year': '2019', 'period': 'M10', 'periodName': 'October', 'value': '4.0', 'footnotes': [{}]}, {'year': '2019', 'period': 'M09', 'periodName': 'September', 'value': '3.2', 'footnotes': [{}]}, {'year': '2019', 'period': 'M08', 'periodName': 'August', 'value': '3.6', 'footnotes': [{}]}, {'year': '2019', 'period': 'M07', 'periodName': 'July', 'value': '3.8', 'footnotes': [{}]}, {'year': '2019', 'period': 'M06', 'periodName': 'June', 'value': '4.0', 'footnotes': [{}]}, {'year': '2019', 'period': 'M05', 'periodName': 'May', 'value': '3.2', 'footnotes': [{}]}, {'year': '2019', 'period': 'M04', 'periodName': 'April', 'value': '4.7', 'footnotes': [{}]}, {'year': '2019', 'period': 'M03', 'periodName': 'March', 'value': '5.2', 'footnotes': [{}]}, {'year': '2019', 'perio

In [4]:
# Date index from first series
date_list = [f"{i['year']}-{i['period'][1:]}-01" for i in p[0]['data']]
# Empty dataframe to fill with values
df = pd.DataFrame()
# Build a pandas series from the API results, p
for s in p:
    df[series_dict[s['seriesID']]] = pd.Series(
        index = pd.to_datetime(date_list),
        data = [i['value'] for i in s['data']]
        ).astype(float).iloc[::-1]
# Show last 5 results
df.tail()

,"(Unadj) Unemployment Rate - Construction Industry, Private Wage and Salary Workers)","(Unadj) Unemployment Rate - Nonagricultural Private Wage and Salary Workers, Educational Services","(Unadj) Unemployment Rate - Nonagricultural Private Wage and Salary Workers, Health Care and Social Assistance","(Unadj) Unemployment Rate - Mining, quarrying, and oil and gas extraction, Nonagricultural Private Wage and Salary Workers","(Unadj) Unemployment Rate - Nonagricultural Private Wage and Salary Workers, Finance","(Unadj) Unemployment Rate - Nonagricultural Private Wage and Salary Workers, Real Estate","(Unadj) Unemployment Rate - Nonagricultural Private Wage and Salary Workers, Insurance","(Unadj) Unemployment Rate - Nonagricultural Private Wage and Salary Workers, Finance and Insurance"
2019-08-01,3.6,4.5,2.3,1.1,2.0,2.3,2.2,2.1
2019-09-01,3.2,2.6,2.2,1.7,1.7,2.4,2.0,1.8
2019-10-01,4.0,2.5,2.1,2.8,1.7,3.0,1.7,1.7
2019-11-01,4.4,3.0,2.1,3.2,2.0,2.0,2.5,2.2
2019-12-01,5.0,3.4,2.1,3.8,1.6,3.4,2.6,2.0


In [5]:
df.dtypes

(Unadj) Unemployment Rate - Construction Industry, Private Wage and Salary Workers)                                           float64
(Unadj) Unemployment Rate - Nonagricultural Private Wage and Salary Workers, Educational Services                             float64
(Unadj) Unemployment Rate - Nonagricultural Private Wage and Salary Workers, Health Care and Social Assistance                float64
(Unadj) Unemployment Rate - Mining, quarrying, and oil and gas extraction, Nonagricultural Private Wage and Salary Workers    float64
(Unadj) Unemployment Rate - Nonagricultural Private Wage and Salary Workers, Finance                                          float64
(Unadj) Unemployment Rate - Nonagricultural Private Wage and Salary Workers, Real Estate                                      float64
(Unadj) Unemployment Rate - Nonagricultural Private Wage and Salary Workers, Insurance                                        float64
(Unadj) Unemployment Rate - Nonagricultural Private Wage and S